In [0]:
dbutils.widgets.dropdown(name = 'environment', choices= ['dev', 'uat', 'prod'], defaultValue = 'dev', label = 'select your environment')
environment = dbutils.widgets.get('environment')

In [0]:
%run "/Workspace/Users/dakuikui_outlook.com#ext#@dakuikuioutlook.onmicrosoft.com/04_common"

In [0]:
df_silvertraffic = spark.readStream\
                    .format('delta').table(f"`{environment}_catalog`.`silver`.`silver_traffic`")

In [0]:
df_silverroads = spark.readStream\
                    .format('delta').table(f"`{environment}_catalog`.`silver`.`silver_roads`")

In [0]:
from pyspark.sql.functions import col, current_timestamp

In [0]:
def add_vehicle_indensity(df):
    print('adding column vehicle indensity: ', end='')
    processed_df = df.withColumn('Vehicle_Indensity', col('Motor_Vehicles_Count')/col('Link_length_km'))
    print('ok')
    return processed_df

def add_loading_time(df):
    print('adding column loading time: ', end='')
    processed_df = df.withColumn('Loading_Time', current_timestamp())
    print('ok')
    return processed_df

In [0]:
def write_gold_traffic(df,environment):
    df.writeStream\
            .format('delta')\
            .option('checkpointLocation', f'{checkpoints}/Gold_Traffic_load/checkpoint')\
            .trigger(availableNow=True)\
            .outputMode('append')\
            .toTable(f'`{environment}_catalog`.`gold`.`gold_traffic`')
    print('successfully write silver traffic to gold table')

def write_gold_roads(df,environment):
    df.writeStream\
            .format('delta')\
            .option('checkpointLocation', f'{checkpoints}/Gold_Roads_load/checkpoint')\
            .trigger(availableNow=True)\
            .outputMode('append')\
            .toTable(f'`{environment}_catalog`.`gold`.`gold_roads`')
    print('successfully write silver roads to gold table')

In [0]:
processed_df1 = add_vehicle_indensity(df_silvertraffic)
processed_df2 = add_loading_time(processed_df1)
write_gold_traffic(processed_df2,environment)
write_gold_roads(df_silverroads,environment)